In [2]:
print("Hello")

Hello


# install libraries

In [11]:
# pip install apify-client
# !pip install textblob
# pip install Praw
# pip install TikTokApi 

# Extract data from reddddit

In [48]:
import praw
import pandas as pd
reddit = praw.Reddit(
    client_id="",
    client_secret="",
    user_agent="my_app"
)

# Example: fetch top posts from r/electronic_cigarette
subreddit = reddit.subreddit("VapeWild")

rows = []
for post in subreddit.top(limit=50):
    row = {
        "datetime": post.created_utc,
        "niche": "vape",
        "product": None,
        "hashtags": None,
        "description": post.title,
        "country": None,
        "area": None,
        "store/channel_name": post.author.name if post.author else None,
        "social_platform": "reddit"
    }
    rows.append(row)

--- make DataFrame ---
df = pd.DataFrame(rows)

# --- convert datetime to readable format ---
df["datetime"] = pd.to_datetime(df["datetime"], unit="s")

# --- show the DataFrame ---
df.head(10)  # print first 10 rows
    


,datetime,niche,product,hashtags,description,country,area,store/channel_name,social_platform
0,2018-11-11 18:04:40,vape,None,None,Vapewild i love you but we need to talk....,None,None,the-jedi,reddit
1,2020-09-10 14:50:13,vape,None,None,So Long and Thanks for All the Fish,None,None,VapeWild,reddit
2,2019-03-23 23:12:28,vape,None,None,7 bottles for $1.10 bless vape wild,None,None,thecrocobear,reddit
3,2018-12-06 20:32:56,vape,None,None,Thanks vapewild!,None,None,Jugglo55,reddit
4,2018-11-20 20:04:48,vape,None,None,Awesome customer service !,None,None,Twrecks_,reddit
5,2018-08-04 13:11:51,vape,None,None,"I love Vape Wild, but there needs to be a way ...",None,None,RandomHero27,reddit
6,2019-09-11 20:24:08,vape,None,None,"In light of recent news, here's a thead for yo...",None,None,VapeWild,reddit
7,2019-01-01 05:31:38,vape,None,None,Just another amazing customer service post.,None,None,captured3,reddit
8,2018-11-14 20:12:14,vape,None,None,"Ordered 60mL, Received 140mL",None,None,Khryzis,reddit
9,2018-10-01 14:36:06,vape,None,None,"Oh boy, we got a lot of stuff today to look at...",None,None,VapeWild,reddit


In [62]:
import pandas as pd
import praw
import re
from collections import Counter
import nltk

# First time only:
# nltk.download("stopwords")
# nltk.download("punkt")
# nltk.download("averaged_perceptron_tagger")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag

stop_words = set(stopwords.words("english"))

def extract_hashtags(text):
    return re.findall(r"#\w+", text)

def extract_candidates(text):
    """
    Extract both brand-like names and flavor names:
    - Brands: capitalized words / 2-word combos
    - Flavors: adjective+noun combos (e.g. 'strawberry kiwi')
    """
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)
    candidates = []

    # --- brand-like names ---
    for i, word in enumerate(tokens):
        if word[0].isupper() and word.lower() not in stop_words:
            if i + 1 < len(tokens) and tokens[i+1][0].isupper():
                candidates.append(word + " " + tokens[i+1])
            else:
                candidates.append(word)

    # --- flavor-like combos (adj + noun) ---
    for i in range(len(tagged)-1):
        w1, t1 = tagged[i]
        w2, t2 = tagged[i+1]
        if t1.startswith("JJ") and t2.startswith("NN"):  # adjective + noun
            phrase = f"{w1} {w2}"
            if phrase.lower() not in stop_words:
                candidates.append(phrase)

    return candidates

rows = []
sub_reddit = "VapeWild"
subreddit = reddit.subreddit("VapeWild")
for post in subreddit.top(limit=50):
    desc = post.title + " " + (post.selftext or "")

    products_flavors = extract_candidates(desc)
    product = products_flavors[0] if products_flavors else None

    row = {
        "datetime": post.created_utc,
        "niche": "vape",
        "product_or_flavor": product,
        "hashtags": extract_hashtags(desc),
        "description": post.title,
        "country": None,
        "area": None,
        "store/channel_name": sub_reddit,
        "social_platform": "reddit"
    }
    rows.append(row)

df = pd.DataFrame(rows)
df["datetime"] = pd.to_datetime(df["datetime"], unit="s")

# Show top discovered brand/flavor names
all_names = [p for p in df["product_or_flavor"].dropna()]
counts = Counter(all_names)
print("Top detected brand/flavor names:", counts.most_common(10))

df.head(10)

Top detected brand/flavor names: [('Vapewild', 4), ('VW', 3), ('VapeWild', 3), ('Vape Wild', 2), ('Long', 1), ('long ass', 1), ('Thanks', 1), ('Awesome', 1), ('recent news', 1), ('Selfishly I', 1)]


,datetime,niche,product_or_flavor,hashtags,description,country,area,store/channel_name,social_platform
0,2018-11-11 18:04:40,vape,Vapewild,[],Vapewild i love you but we need to talk....,None,None,the-jedi,reddit
1,2020-09-10 14:50:13,vape,Long,[],So Long and Thanks for All the Fish,None,None,VapeWild,reddit
2,2019-03-23 23:12:28,vape,long ass,[],7 bottles for $1.10 bless vape wild,None,None,thecrocobear,reddit
3,2018-12-06 20:32:56,vape,Thanks,[],Thanks vapewild!,None,None,Jugglo55,reddit
4,2018-11-20 20:04:48,vape,Awesome,[],Awesome customer service !,None,None,Twrecks_,reddit
5,2018-08-04 13:11:51,vape,Vape Wild,[],"I love Vape Wild, but there needs to be a way ...",None,None,RandomHero27,reddit
6,2019-09-11 20:24:08,vape,recent news,[],"In light of recent news, here's a thead for yo...",None,None,VapeWild,reddit
7,2019-01-01 05:31:38,vape,Selfishly I,"[#x200B, #x200B]",Just another amazing customer service post.,None,None,captured3,reddit
8,2018-11-14 20:12:14,vape,Ordered,[#x200B],"Ordered 60mL, Received 140mL",None,None,Khryzis,reddit
9,2018-10-01 14:36:06,vape,Oh,[#x200B],"Oh boy, we got a lot of stuff today to look at...",None,None,VapeWild,reddit


In [75]:
subreddit = reddit.subreddit("Dubai")

keywords = ["vape", "vape flavour", "e-cigarette", "eliquid"]
query = " OR ".join(keywords)   # builds "vape OR vape flavour OR e-cigarette OR eliquid"

rows = []
for post in subreddit.search(query, limit=100):
    desc = post.title + " " + (post.selftext or "")

    row = {
        "datetime": post.created_utc,
        "niche": None,
        "flavour": None,
        "description": post.title,
        "country": "UAE",
        "store/channel_name": "Dubai",
        "social_platform": "reddit",
        "store/channel_name": post.author.name if post.author else None,
        "social_platform": "reddit"
    }
    rows.append(row)

df = pd.DataFrame(rows)
df["datetime"] = pd.to_datetime(df["datetime"], unit="s")

df.head(10)


,datetime,niche,flavour,description,country,store/channel_name,social_platform
0,2024-10-16 09:34:22,None,None,why has mcdonalds UAE never changed McFlurry f...,UAE,Calm_Strength3266,reddit
1,2025-08-05 10:37:56,None,None,Vapes at Dubai airport,UAE,MoneySeries4403,reddit
2,2025-05-12 18:08:57,None,None,Is it ok to bring vapes in dubai as a 16 y/o,UAE,FewSection958,reddit
3,2025-05-29 07:38:40,None,None,Hi wanted to know about carrying THC vape tran...,UAE,Internal_Fold_2444,reddit
4,2025-06-29 17:45:24,None,None,Vaping inside MOE…pe,UAE,Wlangname,reddit
5,2025-05-26 17:12:27,None,None,Are there vapes sold in Dubai airport?,UAE,debballs,reddit
6,2023-01-10 12:31:53,None,None,What do you think about the increasing Vape Cu...,UAE,JohNdiCk2003,reddit
7,2024-10-30 17:11:47,None,None,Countries with Highest % of E-Cigarette Smokers,UAE,rookieking11,reddit
8,2025-05-10 12:37:33,None,None,Anyone know a coffee that taste similar to thi...,UAE,dedeotaku,reddit
9,2022-02-10 04:15:19,None,None,Is it really illegal to vape in UAE?,UAE,Agitated-Ability-711,reddit


### Using API to dynmically detect the data

In [3]:
import pandas as pd
import praw
import requests
import json
import praw
import pandas as pd
reddit = praw.Reddit(
    client_id="",
    client_secret="",
    user_agent="my_app"
)
subreddit = reddit.subreddit("Dubai")

# vape-related keywords
keywords = ["vape", "vape flavour", "berry blast", "mango", "strawberry"]
query = " OR ".join(keywords)

API_URL = "http://127.0.0.1:8000/detect-product"

def get_niche_and_flavor(text):
    """Send description text to detection API"""
    try:
        payload = {"text": text}
        response = requests.post(API_URL, json=payload, timeout=10)
        if response.status_code == 200:
            data = response.json()
            return data.get("detected_niche"), data.get("detected_product_type")
        else:
            return None, None
    except Exception as e:
        print("API error:", e)
        return None, None

rows = []
for post in subreddit.search(query, limit=50):
    desc = post.title + " " + (post.selftext or "")

    niche, flavor = get_niche_and_flavor(desc)

    row = {
        "datetime": post.created_utc,
        "niche": niche,
        "flavour_or_product": flavor,
        "description": desc,
        "country": "UAE",
        "store/channel_name": post.author.name if post.author else None,
        "social_platform": "reddit"
    }
    rows.append(row)

# Convert to dataframe
df = pd.DataFrame(rows)
df["datetime"] = pd.to_datetime(df["datetime"], unit="s")

API error: HTTPConnectionPool(host='127.0.0.1', port=8000): Read timed out. (read timeout=10)
API error: HTTPConnectionPool(host='127.0.0.1', port=8000): Read timed out. (read timeout=10)


In [4]:
df.head(10)

,datetime,niche,flavour_or_product,description,country,store/channel_name,social_platform
0,2023-04-12 09:31:17,vape,strawberry,does anyone know why all companies have stoppe...,UAE,m_imraan,reddit
1,2025-02-27 19:51:55,vape,mango,If we are doing so well then why is there no p...,UAE,Homoaeternus,reddit
2,2024-10-16 09:34:22,food,McFlurry,why has mcdonalds UAE never changed McFlurry f...,UAE,Calm_Strength3266,reddit
3,2025-08-05 10:37:56,vape,tobacco,Vapes at Dubai airport Hi\nI'm travelling to t...,UAE,MoneySeries4403,reddit
4,2023-06-18 01:48:09,None,None,Tech workers blast startup CEO who said he did...,UAE,sosoricoc000,reddit
5,2021-05-31 11:49:07,None,None,Ahh we love to see it.. its Mango Season,UAE,MisterCrazyGuy,reddit
6,2025-05-12 18:08:57,vape,Cig Bay vape with a battery,Is it ok to bring vapes in dubai as a 16 y/o I...,UAE,FewSection958,reddit
7,2025-05-29 07:38:40,vape,THC vape,Hi wanted to know about carrying THC vape tran...,UAE,Internal_Fold_2444,reddit
8,2020-08-05 10:03:42,None,None,Beirut blast:Dubai's burj Khalifa lights up in...,UAE,silhouetteweirdo,reddit
9,2025-06-29 17:45:24,vape,None,Vaping inside MOE…pe Saw someone vaping inside...,UAE,Wlangname,reddit


In [72]:
df.to_csv('check.csv', index=False)

# Extract relevant information only 

In [8]:
# ----- PRAW: targeted vape data fetch (last 7 days) -----
import re
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo

import praw

# --- 1) Configure your Reddit credentials ---
# reddit = praw.Reddit(
#     client_id="YOUR_CLIENT_ID",
#     client_secret="YOUR_CLIENT_SECRET",
#     user_agent="my_vape_app/1.0"
# )
reddit = praw.Reddit(
    client_id="h7SCVUWMMkrKxa6dW7K6aA",
    client_secret="ndIj2lKITQBJ6NzHAD10FkyuPZer2A",
    user_agent="my_app"
)

# --- 2) Target subreddits that are actually about vaping ---
VAPE_SUBS = [
         "VapeWild","Juul" # remove subs you don't want
]

# --- 3) Vape keyword filter (title/selftext) ---
KEYWORD_RE = re.compile(
    r"\b(vape|vaping|e-?cig|e\s*cigarette|e-?liquid|ejuice|"
    r"nic(?:otine)?\s*salt|pod(?:\s*system)?|disposable|mod|coil|atomizer|"
    r"lush\s*ice|strawberry\s*kiwi|blue\s*razz|mango\s*ice|elf\s*bar|juul|smok|vaporesso|relx|lost\s*mary)\b",
    flags=re.IGNORECASE
)

# --- 4) Simple flavor/product extraction (extend as you wish) ---
FLAVORS = [
    "Lush Ice", "Strawberry Kiwi", "Blue Razz", "Mango Ice", "Watermelon Ice",
    "Peach Ice", "Grape Ice", "Cool Mint", "Triple Mango", "Pink Lemonade", "Cola Ice"
]
BRANDS = ["Elf Bar", "JUUL", "SMOK", "Vaporesso", "RELX", "HQD", "Lost Mary", "Geek Bar", "OXBAR"]

def extract_product(text):
    hit = None
    for f in FLAVORS:
        if f.lower() in text.lower():
            hit = f
            break
    if not hit:
        for b in BRANDS:
            if b.lower() in text.lower():
                hit = b
                break
    return hit

def extract_hashtags(text):
    # Reddit rarely uses hashtags, but capture them if present
    return re.findall(r"#\w+", text)

def to_pk_time(utc_ts):
    dt = datetime.fromtimestamp(utc_ts, tz=timezone.utc).astimezone(ZoneInfo("Asia/Karachi"))
    return dt.strftime("%d/%m/%Y %H:%M:%S")  # matches your 23/08/2025 style

def fetch_reddit_vape_last_week(limit_per_sub=200):
    now_utc = datetime.now(timezone.utc)
    since_utc = now_utc - timedelta(days=7)
    since_ts = since_utc.timestamp()

    all_rows = []
    seen_ids = set()

    # Lucene query focuses search; we still post-filter with regex
    lucene_query = (
        '(title:(vape OR vaping OR "e cig" OR "e-cig" OR "e liquid" OR ejuice OR disposable OR "nic salt" OR pod)'
        ' OR selftext:(vape OR vaping OR "e cig" OR "e-cig" OR "e liquid" OR ejuice OR disposable OR "nic salt" OR pod))'
    )

    for sub in VAPE_SUBS:
        sr = reddit.subreddit(sub)
        # Search recent; we’ll still enforce the exact time range ourselves:
        for post in sr.search(lucene_query, syntax="lucene", sort="new", limit=limit_per_sub):
            if post.id in seen_ids:
                continue
            seen_ids.add(post.id)

            # Time window: last 7 days
            if not getattr(post, "created_utc", None):
                continue
            if post.created_utc < since_ts:
                continue

            # Text to filter against
            title = post.title or ""
            selftext = getattr(post, "selftext", "") or ""
            combined = f"{title}\n{selftext}"

            # Vape keyword filter
            if not KEYWORD_RE.search(combined):
                continue

            # Build row in your schema
            row = {
                "datetime": to_pk_time(post.created_utc),
                "niche": "vape",
                "product": extract_product(combined),
                "hashtags": extract_hashtags(combined),
                "description": title if len(title) > 5 else (selftext[:180] + "…"),
                "country": None,   # Reddit doesn't provide reliably
                "area": None,
                "store/channel_name": post.author.name if post.author else None,
                "social_platform": "reddit",
            }
            all_rows.append(row)

    return all_rows

if __name__ == "__main__":
    rows = fetch_reddit_vape_last_week()
    for r in rows[:10]:
        print(r)
    print(f"Total rows: {len(rows)}")


Total rows: 0


# Extract data from TikTok

In [24]:
from TikTokApi import TikTokApi

In [27]:
pip install tiktokapipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
from TikTokApi import TikTokApi